In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 4.6 MB/s 
     |████████████████████████████████| 895 kB 37.2 MB/s 
     |████████████████████████████████| 6.8 MB 26.9 MB/s 
     |████████████████████████████████| 596 kB 32.9 MB/s 
     |████████████████████████████████| 67 kB 3.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd 
import numpy as np 
import os
import torch
import torch.nn as nn

import warnings 
warnings.filterwarnings("ignore")
from tqdm import tqdm
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from transformers.optimization import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup
import re
from sklearn.model_selection import train_test_split

In [ ]:
# Random Seed Fix
import random
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = True  
seed_everything()

In [ ]:
device = torch.device("cuda")
!nvidia-smi

Sat Feb 19 07:15:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
############# HYPERPARMS ##############
num_epochs = 7
batch_size =128
lr = 0.00001
pretrain = "monologg/koelectra-base-v3-discriminator"

In [ ]:
def load_data(path):
  TRAIN = os.path.join(path, 'benchmark_train_data.csv')
  TEST = os.path.join(path, 'test_data.csv')
  SS = os.path.join(path, 'sample_submission.csv')
  label_dict = {"entailment" : 0, "contradiction" : 1, "neutral" : 2}
  train = pd.read_csv(TRAIN)
  test = pd.read_csv(TEST)
  sample_submission = pd.read_csv(SS)
  train['label'] = train['label'].map(label_dict)

  return train,test,sample_submission

def text_clean(df):
  df["premise_"] = "[CLS]" + df["premise"].astype(str) + "[SEP]"
  df["hypothesis_"] = df["hypothesis"].astype(str) + "[SEP]"
  df["text_sum"] = df.premise_ + " " + df.hypothesis_
  df = df[['text_sum','label']]
  return df 

def random_deletion(sentence, p=0.2):
    words = sentence.split ()
    n = len (words)
    if n == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words))
    #print (remaining) 
    if len(remaining) == 0: # if not left, sample a random word
        return ' '.join ([random.choice(words)])
    else:
        return ' '.join (remaining)

def random_swap(sentence, n=2):
    sentence = sentence.split () 
    length = range(len(sentence))
    swapped = []
    if len(sentence) >2:
      for _ in range(n):
          idx1, idx2 = random.sample(length, 2)
          swapped.append ([sentence[idx1], sentence[idx2]])
          sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return ' '.join (sentence)

def eda_aug(df):

    cache = {'premise':[], 'hypothesis':[], 'label':[]}
    for idx in tqdm(range(len(df))):
        premise = df.iloc[idx]['premise']
        hypothesis = df.iloc[idx]['hypothesis']
        label = df.iloc[idx]['label']
        cache['premise'].append(premise)
        cache['hypothesis'].append(hypothesis)
        cache['label'].append(label)
        #cache['premise'].append(random_deletion(premise))
        #cache['hypothesis'].append(random_deletion(hypothesis))
        #cache['label'].append(label)
        cache['premise'].append(random_swap(premise))
        cache['hypothesis'].append(random_swap(hypothesis))
        cache['label'].append(label)
    
    return pd.DataFrame(cache)



#ROOT = '/content/drive/MyDrive/DACON_MONTHLYNLI'
ROOT = '/content/drive/Shareddrives/Dacon/data'
train,test,sample_submission = load_data(ROOT)
###### AUGMENTATION ######
train = eda_aug(train)
###### AUGMENTATION ######

clean_train,clean_test  = text_clean(train),text_clean(test)
display(clean_train)

100%|██████████| 27998/27998 [00:16<00:00, 1662.77it/s]


,text_sum,label
0,"[CLS]씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나...",1
1,"[CLS]씨름은 전해져 내려오는 남자들의 대표적인 놀이로서, 장정들이 평평한 백사장...",1
2,"[CLS]씨름은 상고시대로부터 전해져 내려오는 놀이로서, 대표적인 남자들의 소년이나...",1
3,[CLS]삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 ...,1
4,[CLS]자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나...,1
...,...,...
83989,[CLS]발코니가 있는 방이면 발코니에서 흡연이 가능합니다.[SEP] 비흡연자는 발...,2
83990,[CLS]흡연자분들은 있는 흡연이 방이면 발코니에서 발코니가 가능합니다.[SEP] ...,2
83991,[CLS]흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.[SEP] ...,0
83992,[CLS]흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.[SEP] ...,0


In [ ]:
############# Dataset ##############
class CustomDataset(Dataset):
  
  def __init__(self,dataset,option):
    
    self.dataset = dataset 
    self.option = option
    self.tokenizer = AutoTokenizer.from_pretrained(pretrain)

  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:2].values
    text = row[0]
    #y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=70,
        pad_to_max_length=True,
        add_special_tokens=False
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]
    
    if self.option =='train':
        y =row[1]
        return input_ids,attention_mask,y

    return input_ids, attention_mask

In [ ]:
############### CV ################
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 1,shuffle=True,random_state=42)
folds=[]
for trn_idx,val_idx in skf.split(clean_train['text_sum'],clean_train['label']):
    folds.append((trn_idx,val_idx))

ValueError: ignored

In [ ]:
### NON - CV ###
best_models = []

model = ElectraForSequenceClassification.from_pretrained(pretrain,num_labels=3).to(device)
model=nn.DataParallel(model).to(device)
optimizer = AdamW(model.parameters(), lr=lr)

random_idx = random.sample(range(len(clean_train)), len(clean_train))
train_idx = random_idx[:48000]
val_idx = random_idx[48000:]
print(len(train_idx), len(val_idx))
print(train_idx[:5], val_idx[:5])
train_data = clean_train.iloc[train_idx]
val_data = clean_train.iloc[val_idx]

train_dataset = CustomDataset(train_data,'train')
valid_dataset = CustomDataset(val_data,'train')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

warmup_ratio = 0.1
total_steps = len(train_loader) * num_epochs
warmup_step = int(total_steps * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1, num_training_steps=total_steps)
valid_loss_min = 0.4
valid_acc_max = 0.8

for epoch in range(num_epochs):
    batches = 0
    total_loss = 0.0
    correct = 0
    total =0
    model.train()
    
    for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()
        y_batch = y_batch.to(device)
        y_pred = model(input_ids_batch.to(device), attention_mask = attention_masks_batch.to(device))[0]
        #print(y_pred)
        #loss1 = F.cross_entropy(y_pred, y_batch)
        #loss2= F.hinge_embedding_loss(y_pred, y_batch)
        #loss = 0.7*loss1+0.3*loss2
        loss1 = F.cross_entropy(y_pred, y_batch)
        one_hot = torch.nn.functional.one_hot(y_batch, 3).to(torch.float32)
        loss2 = F.mse_loss(y_pred, one_hot)
        loss3= F.hinge_embedding_loss(y_pred, one_hot)
        loss = 0.7*loss1+0.15*loss2+0.15*loss3
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum()
        total += len(y_batch)
        batches += 1
        if batches % 100 == 0:
            print("Batch Loss: ", total_loss, "Accuracy: ", correct.float() / total)
  
    val_loss = []
    val_acc = []
    
    for input_ids_batch, attention_masks_batch, y_batch in tqdm(valid_loader):
        
        model.eval()
        with torch.no_grad():
            
            y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
            valid_loss = F.cross_entropy(y_pred,y_batch.to(device)).cpu().detach().numpy()

            preds = torch.argmax(y_pred,1)
            preds = preds.cpu().detach().numpy()
            y_batch = y_batch.cpu().detach().numpy()
            batch_acc = (preds==y_batch).mean()
            val_loss.append(valid_loss)
            val_acc.append(batch_acc)
            
            
    val_loss = np.mean(val_loss)
    val_acc = np.mean(val_acc)
    scheduler.step()
    print(f'Epoch: {epoch} - valid Loss: {val_loss:.6f} - valid_acc : {val_acc:.6f}')
    print(optimizer.param_groups[0]["lr"])
    if valid_acc_max < val_acc:
        valid_acc_max = val_acc
        best_models.append(model)
        torch.save(model, f'koelectra-{len(best_models)}.pth') 
        print('model save, model val acc : ',val_acc)
        print('best_models size : ',len(best_models))

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/431M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

72000 11994
[41804, 23754, 66852, 16267, 81229] [60192, 10879, 61640, 18931, 65549]


Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

...

 18%|█▊        | 100/563 [05:02<23:24,  3.03s/it]

Batch Loss:  51.09257361292839 Accuracy:  tensor(0.8900, device='cuda:0')


 36%|███▌      | 200/563 [10:05<18:19,  3.03s/it]

Batch Loss:  102.30954366922379 Accuracy:  tensor(0.8881, device='cuda:0')


 42%|████▏     | 235/563 [11:51<16:26,  3.01s/it]

In [ ]:
best_models = []

for i,fold in enumerate(range(5)):
    print('===============',i+1,'fold start===============')
    model = ElectraForSequenceClassification.from_pretrained(pretrain,num_labels=3).to(device)
    model=nn.DataParallel(model).to(device)
    optimizer = AdamW(model.parameters(), lr=lr)
    
    train_idx = folds[fold][0]
    valid_idx = folds[fold][1]
    train_data = clean_train.loc[trn_idx]
    val_data = clean_train.loc[valid_idx]
    train_dataset = CustomDataset(train_data,'train')
    valid_dataset = CustomDataset(val_data,'train')
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    warmup_ratio = 0.1
    total_steps = len(train_loader) * num_epochs
    warmup_step = int(total_steps * warmup_ratio)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1, num_training_steps=total_steps)
    valid_loss_min = 0.4
    valid_acc_max = 0.8
    
    for epoch in range(num_epochs):
        batches = 0
        total_loss = 0.0
        correct = 0
        total =0
        model.train()
        
        for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
            optimizer.zero_grad()
            y_batch = y_batch.to(device)
            y_pred = model(input_ids_batch.to(device), attention_mask = attention_masks_batch.to(device))[0]
            loss = F.cross_entropy(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            _, predicted = torch.max(y_pred, 1)
            correct += (predicted == y_batch).sum()
            total += len(y_batch)
            batches += 1
            if batches % 100 == 0:
                print("Batch Loss: ", total_loss, "Accuracy: ", correct.float() / total)
      
        val_loss = []
        val_acc = []
        
        for input_ids_batch, attention_masks_batch, y_batch in tqdm(valid_loader):
            
            model.eval()
            with torch.no_grad():
                
                y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
                valid_loss = F.cross_entropy(y_pred,y_batch.to(device)).cpu().detach().numpy()

                preds = torch.argmax(y_pred,1)
                preds = preds.cpu().detach().numpy()
                y_batch = y_batch.cpu().detach().numpy()
                batch_acc = (preds==y_batch).mean()
                val_loss.append(valid_loss)
                val_acc.append(batch_acc)
                
                
        val_loss = np.mean(val_loss)
        val_acc = np.mean(val_acc)
        scheduler.step()
        print(f'Epoch: {epoch} - valid Loss: {val_loss:.6f} - valid_acc : {val_acc:.6f}')
        print(optimizer.param_groups[0]["lr"])
        if valid_acc_max < val_acc:
            valid_acc_max = val_acc
            best_models.append(model)
            torch.save(model.state_dict(), f'/content/drive/Shareddrives/Dacon/saved models/Junha/Electra_Benchmark/koelectra-{len(best_models)}.pth') 
            print('model save, model val acc : ',val_acc)
            print('best_models size : ',len(best_models))

=============== 1 fold start===============


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

Batch Loss:  109.94188272953033 Accuracy:  tensor(0.3420, device='cuda:0')


100%|██████████| 73/73 [01:20<00:00,  1.10s/it]


Epoch: 0 - valid Loss: 1.097843 - valid_acc : 0.355517
1e-05


...

Batch Loss:  6.517204710282385 Accuracy:  tensor(0.9812, device='cuda:0')


100%|██████████| 73/73 [01:20<00:00,  1.10s/it]


Epoch: 9 - valid Loss: 0.023446 - valid_acc : 0.995174
9.999061140516881e-06
model save, model val acc :  0.9951740135815478
best_models size :  17
=============== 3 fold start===============


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

RuntimeError: ignored

In [ ]:
# koelectra-4k번 모델이 가장 성능이 좋은 것으로 가정
test_dataset = CustomDataset(clean_test,'test')
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

preds = dict()
for idx, m in enumerate(best_models):
    #if idx % num_epochs == 2 or idx % num_epochs == 3 or idx % num_epochs == 4:
    if idx == 4:
      print(f'{idx+1} 번째 모델 예측 진행중')
      bestm = m
      bestm.eval()
      answer = []
      with torch.no_grad():
          for input_ids_batch, attention_masks_batch in tqdm(test_loader):
              y_pred = bestm(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0].detach().cpu().numpy()
              answer.extend(y_pred)
      preds[idx+1] = answer

5 번째 모델 예측 진행중


100%|██████████| 27/27 [00:06<00:00,  4.01it/s]


In [ ]:
df = pd.DataFrame()
for key in preds.keys():
  df = pd.concat([df, pd.DataFrame(np.array(preds[key]))], axis =1 )
df.columns = [i for i in range(3*3)]
df

,0,1,2,3,4,5,6,7,8
0,-0.692385,1.794275,-0.602543,-0.692385,1.794275,-0.602543,-0.692385,1.794275,-0.602543
1,-0.048374,-0.775319,1.330322,-0.048374,-0.775319,1.330322,-0.048374,-0.775319,1.330322
2,0.597040,-0.809023,0.740466,0.597040,-0.809023,0.740466,0.597040,-0.809023,0.740466
3,-0.687000,1.872369,-0.674526,-0.687000,1.872369,-0.674526,-0.687000,1.872369,-0.674526
4,-0.447792,1.420751,-0.553400,-0.447792,1.420751,-0.553400,-0.447792,1.420751,-0.553400
...,...,...,...,...,...,...,...,...,...
1661,-0.691068,-0.608387,1.780281,-0.691068,-0.608387,1.780281,-0.691068,-0.608387,1.780281
1662,0.281797,-0.801149,1.092021,0.281797,-0.801149,1.092021,0.281797,-0.801149,1.092021
1663,0.192983,-0.854506,1.201216,0.192983,-0.854506,1.201216,0.192983,-0.854506,1.201216
1664,-0.707620,-0.604393,1.758602,-0.707620,-0.604393,1.758602,-0.707620,-0.604393,1.758602


In [ ]:
### SINGLE PREDICTION FOR CONCORDANCE OBSERVATION ###
single_preds = dict()
concat_probs = pd.DataFrame()
temp = np.zeros((1666, 3))
for key in preds.keys():
  x = np.array(preds[key])
  max = np.max(x,axis=1,keepdims=True) #returns max of each row and keeps same dims
  e_x = np.exp(x - max) #subtracts each row with its max value
  sum = np.sum(e_x,axis=1,keepdims=True) #returns sum of each row and keeps same dims
  f_x = e_x / sum
  #single_preds['prob_'+str(key)] = pd.DataFrame(f_x)
  single_preds['pred_'+str(key)] = pd.DataFrame(np.argmax(f_x, axis=1))
  concat_probs = pd.concat([concat_probs, pd.DataFrame(f_x)], axis =1)
columns = []

for j in range(3,6):
  for i in range(3):
    column = f"{j}_{i}"
    columns.append(column)
concat_probs.columns = columns
display(concat_probs)
concat_probs.to_csv('submission_KoELECTRA_soft.csv', index=False)

,3_0,3_1,3_2,4_0,4_1,4_2,5_0,5_1,5_2
0,0.070846,0.851648,0.077506,0.070846,0.851648,0.077506,0.070846,0.851648,0.077506
1,0.183382,0.088644,0.727975,0.183382,0.088644,0.727975,0.183382,0.088644,0.727975
2,0.416783,0.102156,0.481061,0.416783,0.102156,0.481061,0.416783,0.102156,0.481061
3,0.066933,0.865293,0.067774,0.066933,0.865293,0.067774,0.066933,0.865293,0.067774
4,0.119351,0.773259,0.107390,0.119351,0.773259,0.107390,0.119351,0.773259,0.107390
...,...,...,...,...,...,...,...,...,...
1661,0.071815,0.078005,0.850179,0.071815,0.078005,0.850179,0.071815,0.078005,0.850179
1662,0.278784,0.094395,0.626821,0.278784,0.094395,0.626821,0.278784,0.094395,0.626821
1663,0.244405,0.085741,0.669854,0.244405,0.085741,0.669854,0.244405,0.085741,0.669854
1664,0.072012,0.079843,0.848146,0.072012,0.079843,0.848146,0.072012,0.079843,0.848146


In [ ]:
### ENSEMBLED PREDICTION ###
temp = np.zeros((1666, 3))
for key in preds.keys():
  x = np.array(preds[key])
  max = np.max(x,axis=1,keepdims=True) #returns max of each row and keeps same dims
  e_x = np.exp(x - max) #subtracts each row with its max value
  sum = np.sum(e_x,axis=1,keepdims=True) #returns sum of each row and keeps same dims
  f_x = e_x / sum 
  temp += f_x
temp = temp/3
softvoted_prob = pd.DataFrame(temp)
softvoted_pred = pd.DataFrame(np.argmax(temp, axis=1))
decode_map = {0 : "entailment" , 1 :  "contradiction" , 2 : "neutral" }
sample_submission['label'] = softvoted_pred
sample_submission['label'] = sample_submission['label'].map(decode_map)
sample_submission.to_csv('submission_KoELECTRA_best1.csv', index = False)

In [ ]:
!mkdir /content/drive/Shareddrives/Dacon/saved_models/Junha/Electra_Benchmark_customloss
!mv *.pth /content/drive/Shareddrives/Dacon/saved_models/Junha/Electra_Benchmark_customloss/